In [1]:
!git clone https://github.com/PaddlePaddle/PaddleDetection.git

Cloning into 'PaddleDetection'...
remote: Enumerating objects: 258348, done.
remote: Counting objects: 100% (769/769), done.
remote: Compressing objects: 100% (388/388), done.
remote: Total 258348 (delta 619), reused 381 (delta 381), pack-reused 257579 (from 3)
Receiving objects: 100% (258348/258348), 422.20 MiB | 29.41 MiB/s, done.
Resolving deltas: 100% (210513/210513), done.


In [1]:
!pip install -r PaddleDetection/requirements.txt

In [2]:
!pip install paddlepaddle-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.9/758.9 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: opt-einsum
    Found existing installation: opt_einsum 3.4.0
    Uninstalling opt_einsum-3.4.0:
      Successfully uninstalled opt_einsum-3.4.0


In [3]:
# Video Path
!video_path=/content/TB_F_TB2_4019-2025-07-10_12h08min00s036ms.mp4

In [ ]:
!python /content/PaddleDetection/tools/infer_mot.py \
  -c /content/PaddleDetection/configs/mot/bytetrack/bytetrack_ppyoloe.yml \
  --video_file "{video_path}" \
  --save_videos

[09/05 06:25:12] ppdet.utils.checkpoint INFO: Skipping import of the encryption module.
[09/05 06:25:12] ppdet.data.source.category WARNING: anno_file 'None' is None or not set or not exist, please recheck TrainDataset/EvalDataset/TestDataset.anno_path, otherwise the default categories will be used by metric_type.
[09/05 06:25:12] ppdet.data.source.category WARNING: metric_type: MOT, load default categories of pedestrian MOT.
[09/05 06:25:12] ppdet.utils.download INFO: Downloading ppyoloe_crn_l_36e_640x640_mot17half.pdparams from https://bj.bcebos.com/v1/paddledet/models/mot/ppyoloe_crn_l_36e_640x640_mot17half.pdparams
[09/05 06:25:45] ppdet.utils.checkpoint INFO: Finish resuming model weights: /root/.cache/paddle/weights/ppyoloe_crn_l_36e_640x640_mot17half.pdparams
[09/05 06:32:58] ppdet.data.source.mot INFO: Length of the video: 2168 frames.
[09/05 06:32:58] ppdet.engine.tracker INFO: Starting tracking video /content/TB_F_TB2_4019-2025-07-10_12h08min00s036ms.mp4
[09/05 06:45:59] ppde

W0905 06:25:12.334376 39653 gpu_resources.cc:119] Please NOTE: device: 0, GPU Compute Capability: 7.5, Driver API Version: 12.4, Runtime API Version: 11.8
W0905 06:25:12.340466 39653 gpu_resources.cc:164] device: 0, cuDNN Version: 9.2.
100%|██████████| 2168/2168 [07:15<00:00,  4.98it/s]
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libope

In [ ]:
import subprocess

video_path = "/content/TB_F_TB2_4019-2025-07-10_12h08min00s036ms.mp4"

# Run ffprobe to get the avg_frame_rate as a string (e.g. "30/1" or "1/1")
fps_str = subprocess.check_output([
    "ffprobe", "-v", "error", "-select_streams", "v:0",
    "-show_entries", "stream=avg_frame_rate",
    "-of", "default=noprint_wrappers=1:nokey=1",
    video_path
]).decode().strip()

# Convert to float
num, den = map(int, fps_str.split("/"))
fps = num / den if den != 0 else 0

print("Detected FPS:", fps)


Detected FPS: 12.000199173430264


In [ ]:
output_pattern = "output/mot_outputs/TB_F_TB2_4019-2025-07-10_12h08min00s036ms/%05d.jpg"
output_file = "output/mot_outputs/TB_F_TB2_4019-2025-07-10_12h08min00s036ms_vis_fixedfps.mp4"

subprocess.run([
    "ffmpeg", "-y",
    "-r", str(fps),              # use detected FPS
    "-i", output_pattern,
    "-c:v", "libx264", "-pix_fmt", "yuv420p",
    output_file
])

CompletedProcess(args=['ffmpeg', '-y', '-r', '12.000199173430264', '-i', 'output/mot_outputs/TB_F_TB2_4019-2025-07-10_12h08min00s036ms/%05d.jpg', '-c:v', 'libx264', '-pix_fmt', 'yuv420p', 'output/mot_outputs/TB_F_TB2_4019-2025-07-10_12h08min00s036ms_vis_fixedfps.mp4'], returncode=0)

In [ ]:
%%bash

#CUDA_VISIBLE_DEVICES=0
#python PaddleDetection/tools/infer_mot.py -c PaddleDetection/configs/mot/jde/jde_darknet53_30e_1088x608.yml -o weights=https://paddledet.bj.bcebos.com/models/mot/jde_darknet53_30e_1088x608.pdparams --video_file=fixed_test_r1_c0_x0_y440_20250904T073613.mp4  --save_videos